# 手把手打一场NLP赛事

自然语言处理（Natural Language Processing，NLP）是指计算机处理和理解人类语言的技术。NLP涵盖了从文本的基本语法和词汇处理到更高级的任务，如机器翻译、情感分析、问答系统等。NLP利用计算机算法和模型，对文本进行分词、词性标注、句法分析等处理，以便将人类语言转化为计算机可以理解和处理的形式

## 1. 赛题信息
医学领域的文献库中蕴含了丰富的疾病诊断和治疗信息，如何高效地从海量文献中提取关键信息，进行疾病诊断和治疗推荐，对于临床医生和研究人员具有重要意义。  


基于论文摘要的文本分类与关键词抽取挑战赛  
https://challenge.xfyun.cn/topic/info?type=abstract-of-the-paper&ch=ymfk4uU

![](https://ai-studio-static-online.cdn.bcebos.com/bc8c545638eb4200a68836ed741b6fe7d75108e9009d443b8de5b33fb8e0fa55)


## 2.准备步骤

### 2.1 赛事报名
赛事地址：https://challenge.xfyun.cn/topic/info?type=abstract-of-the-paper&ch=ZuoaKcY
1. 点击报名参赛，登录讯飞开放平台。

### 2.2 数据下载
数据已提前下载在数据集目录下，您可以自行查看其中的train与test文件

### 2.3 环境配置参考资料
python环境的搭建请参考：
- [Mac设备：Mac上安装Anaconda最全教程](https://zhuanlan.zhihu.com/p/350828057)
- [Windows设备：Anaconda超详细安装教程(Windows环境下)_菜鸟1号!!的博客-CSDN博客_windows安装anaconda](https://blog.csdn.net/fan18317517352/article/details/123035625)
Jupyter 编辑器的使用请参考：
- [Jupyter Notebook最全使用教程，看这篇就够了！](https://www.jianshu.com/p/6cc047bc94e5)

## 3. 赛题解析
实践任务
本任务分为两个子任务：
1. 从论文标题、摘要作者等信息，判断该论文是否属于医学领域的文献。
2. 从论文标题、摘要作者等信息，提取出该论文关键词。

第一个任务看作是一个文本二分类任务。机器需要根据对论文摘要等信息的理解，将论文划分为医学领域的文献和非医学领域的文献两个类别之一。第二个任务看作是一个文本关键词识别任务。机器需要从给定的论文中识别和提取出与论文内容相关的关键词。  

**本次学习中我们仅学习第一个任务**  

数据集解析
训练集与测试集数据为CSV格式文件，各字段分别是标题、作者和摘要。Keywords为任务2的标签，label为任务1的标签。训练集和测试集都可以通过pandas读取。

![](https://ai-studio-static-online.cdn.bcebos.com/8c88537bce9d46049151389396c4c5b828556ba332d34ed3a24555e4e28e7191)


## 4.实践思路&baseline
### 实践思路
本赛题任务主要如下：
1. 从论文标题、摘要作者等信息，判断该论文是否属于医学领域的文献。


#### 任务一：文本二分类
第一个任务看作是一个文本二分类任务。机器需要根据对论文摘要等信息的理解，将论文划分为医学领域的文献和非医学领域的文献两个类别之一。

针对文本分类任务，可以提供两种实践思路，一种是使用传统的特征提取方法（如TF-IDF/BOW）结合机器学习模型，另一种是使用预训练的BERT模型进行建模。使用特征提取 + 机器学习的思路步骤如下：
1. 数据预处理：首先，对文本数据进行预处理，包括文本清洗（如去除特殊字符、标点符号）、分词等操作。可以使用常见的NLP工具包（如NLTK或spaCy）来辅助进行预处理。
2. 特征提取：使用TF-IDF（词频-逆文档频率）或BOW（词袋模型）方法将文本转换为向量表示。TF-IDF可以计算文本中词语的重要性，而BOW则简单地统计每个词语在文本中的出现次数。可以使用scikit-learn库的TfidfVectorizer或CountVectorizer来实现特征提取。
3. 构建训练集和测试集：将预处理后的文本数据分割为训练集和测试集，确保数据集的样本分布均匀。
4. 选择机器学习模型：根据实际情况选择适合的机器学习模型，如朴素贝叶斯、支持向量机（SVM）、随机森林等。这些模型在文本分类任务中表现良好。可以使用scikit-learn库中相应的分类器进行模型训练和评估。
5. 模型训练和评估：使用训练集对选定的机器学习模型进行训练，然后使用测试集进行评估。评估指标可以选择准确率、精确率、召回率、F1值等。
6. 调参优化：如果模型效果不理想，可以尝试调整特征提取的参数（如词频阈值、词袋大小等）或机器学习模型的参数，以获得更好的性能。  


Baseline中我们选择使用BOW将文本转换为向量表示，选择逻辑回归模型来完成训练和评估  
代码演示如下：

In [108]:
# 导入pandas用于读取表格数据
import pandas as pd
import random
# 导入BOW（词袋模型），可以选择将CountVectorizer替换为TfidfVectorizer（TF-IDF（词频-逆文档频率）），注意上下文要同时修改，亲测后者效果更佳
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 导入LogisticRegression回归模型
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.model_selection import train_test_split
# 过滤警告消息
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)




In [101]:
# 读取数据集
train = pd.read_csv('data/train.csv')
train['title'] = train['title'].fillna('')
train['abstract'] = train['abstract'].fillna('')
test = pd.read_csv('data/testB.csv')
test['title'] = test['title'].fillna('')
test['abstract'] = test['abstract'].fillna('')
# 提取文本特征，生成训练集与测试集
train['text'] = train['title'].fillna('')  + ' ' +  test['author'].fillna('') + ' ' + train['Keywords'].fillna('')
test['text'] = test['title'].fillna('') + ' ' +  test['author'].fillna('') + ' ' + test['abstract'].fillna('')

In [102]:
#训练集第一条
train['text'][0]

'Accessible Visual Artworks for Blind and Visually Impaired People: Comparing a Multimodal Approach with Tactile Graphics Julia Brox Skranes,Magnus Nakrem Lyngbakken,Kristian Hveem,Helge Røsjø,Torbjørn Omland,Julia Brox Skranes,Magnus Nakrem Lyngbakken,Kristian Hveem,Helge Røsjø,Torbjørn Omland accessibility technology; multimodal interaction; auditory interface; touch interface; vision impairment'

In [103]:
#去掉标点符号
def replace(x):
    s = [',', ';' , '.', '\'','\"' , ':']
    if isinstance(x,str):
        for st in s:
            x = x.replace(st, '')
    return x

train['text'] = train['text'].apply(replace).fillna('')
test['text'] = test['text'].apply(replace).fillna('')
train['text'][0]


'Accessible Visual Artworks for Blind and Visually Impaired People Comparing a Multimodal Approach with Tactile Graphics Julia Brox SkranesMagnus Nakrem LyngbakkenKristian HveemHelge RøsjøTorbjørn OmlandJulia Brox SkranesMagnus Nakrem LyngbakkenKristian HveemHelge RøsjøTorbjørn Omland accessibility technology multimodal interaction auditory interface touch interface vision impairment'

In [104]:
vector = TfidfVectorizer().fit(train['text'])
train_vector = vector.transform(train['text'])
test_vector = vector.transform(test['text'])

In [105]:

# 引入模型
model = LogisticRegression()
# 开始训练，这里可以考虑修改默认的batch_size与epoch来取得更好的效果
model.fit(train_vector, train['label'])
# 利用模型对测试集label标签进行预测
test['label'] = model.predict(test_vector)
test['Keywords'] = test['title'].fillna('')
test[['uuid','Keywords','label']].to_csv('submit_task1_linear.csv', index=None)

In [110]:
clf = tree.DecisionTreeClassifier(criterion = "entropy"
                                  ,random_state = random.randint(10,50) #每次选取的特征不一样，使用random_state来固定选取的特征
                                  #splitter选取哪个特征作为内部节点，best的重要性最大的最为内部节点，而random是采用随机的方式进行选取
                                  ,splitter = "random"
                                  )
clf = clf.fit(train_vector,train['label'])
score = clf.score(train_vector,train['label'])#return the predition



0.6801666666666667

In [111]:
from sklearn.metrics import accuracy_score
accuracy_score(clf.predict(train_vector),train['label'])

0.6801666666666667

In [114]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=128, max_iter=20, alpha=1e-4,
                    solver='adam', verbose=10, random_state=42, tol=1e-4,
                    learning_rate_init=0.001)
mlp = mlp.fit(train_vector,train['label'])


Iteration 1, loss = 0.68663722
Iteration 2, loss = 0.63868379
Iteration 3, loss = 0.57076912
Iteration 4, loss = 0.52208494
Iteration 5, loss = 0.49619660
Iteration 6, loss = 0.48314873
Iteration 7, loss = 0.47649102
Iteration 8, loss = 0.47233993
Iteration 9, loss = 0.47057245
Iteration 10, loss = 0.46859999
Iteration 11, loss = 0.46729358
Iteration 12, loss = 0.46657006
Iteration 13, loss = 0.46618172
Iteration 14, loss = 0.46534145
Iteration 15, loss = 0.46496924
Iteration 16, loss = 0.46463559
Iteration 17, loss = 0.46494813
Iteration 18, loss = 0.46443958
Iteration 19, loss = 0.46444379
Iteration 20, loss = 0.46412110


In [126]:
score = mlp.score(train_vector,train['label'])#return the predition
# 利用模型对测试集label标签进行预测
test['label'] = mlp.predict(test_vector)
test['Keywords'] = test['title'].fillna('')
test[['uuid','Keywords','label']].to_csv('submit_task1_mlp1.csv',index=False)

In [128]:
accuracy_score(mlp.predict(train_vector),train['label'])

0.6801666666666667